In [1]:
!pip install -q flwr torch torchvision tensorboard
!pip install -U "flwr[simulation]"
!pip install timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.0/540.0 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datetime import datetime
from functools import partial
from flwr.server import ServerConfig
import random
import numpy as np
import torch.nn as nn
from torchvision import transforms
import timm
import torch
import os
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split, DataLoader
from torch.utils.tensorboard import SummaryWriter
import flwr as fl
from flwr.simulation import run_simulation
from flwr.common import Context
from flwr.server import ServerApp, ServerAppComponents, ServerConfig
from flwr.client import ClientApp
from flwr.common import parameters_to_ndarrays, ndarrays_to_parameters
from flwr.common import Parameters
from collections import OrderedDict
from wandb_logger import FederatedWandBLogger
import data_utils
import clients
import strategies
import data_preprocessing

In [3]:
def build_optimizer_config(optimizer_type):
    if optimizer_type == clients.OptimizerType.SGD:
        config = {
            "lr": LR,
            "momentum": MOMENTUM,
            "weight_decay": WEIGHT_DECAY,
            "nesterov": NESTEROV
        }

    elif optimizer_type == clients.OptimizerType.SSGD:
        config = {
            "lr": LR,
            "momentum": MOMENTUM,
            "weight_decay": WEIGHT_DECAY,
        }


    elif optimizer_type == clients.OptimizerType.ADAM:
        config = {
            "lr": LR,
            "betas": BETAS,
            "weight_decay": WEIGHT_DECAY,
            "eps": EPSILON
        }

    elif optimizer_type == clients.OptimizerType.ADAMW:
        config = {
            "lr": LR,
            "betas": BETAS,
            "weight_decay": WEIGHT_DECAY
        }

    elif optimizer_type == clients.OptimizerType.SADAMW:
        config = {
            "lr": LR,
            "betas": BETAS,
            "weight_decay": WEIGHT_DECAY,
            "eps": EPSILON
        }

    elif optimizer_type == clients.OptimizerType.RMSPROP:
        config = {
            "lr": LR,
            "alpha": ALPHA,
            "eps": EPSILON,
            "weight_decay": WEIGHT_DECAY,
            "momentum": MOMENTUM,
            "centered": CENTERED
        }

    elif optimizer_type == clients.OptimizerType.ADAGRAD:
        config = {
            "lr": LR,
            "weight_decay": WEIGHT_DECAY,
            "eps": EPSILON
        }
    else:
        raise ValueError(f"Unsupported optimizer type: {optimizer_type}")

    print(f"Optimizer '{optimizer_type}' initialized successfully.")
    return config

In [4]:
def build_scheduler_config(scheduler_type):
    scheduler_type.lower()

    if scheduler_type == "cosine":
        config = {
            "T_max": T_MAX,
            "eta_min": ETA_MIN
        }

    elif scheduler_type == "cosine_restart":
        config = {
            "T_0": T_MAX,
            "T_mult": 1,
            "eta_min": ETA_MIN
        }

    elif scheduler_type == "step":
        config = {
            "step_size": STEP_SIZE,
            "gamma": GAMMA
        }

    elif scheduler_type == "multistep":
        config = {
            "milestones": MILESTONES,
            "gamma": GAMMA
        }

    elif scheduler_type == "exponential":
        config = {
            "gamma": GAMMA
        }

    elif scheduler_type == "reduce_on_plateau":
        config = {
            "mode": "min",
            "factor": FACTOR,
            "patience": PATIENCE,
            "threshold": THRESHOLD
        }

    elif scheduler_type == "constant":
        config = {
            "factor": 1.0,
            "total_iters": TOTAL_ITERS
        }

    elif scheduler_type == "linear":
        config = {
            "start_factor": 0.5,
            "end_factor": LR_END,
            "total_iters": TOTAL_ITERS
        }
    else:
        raise ValueError(f"Unsupported scheduler type: {scheduler_type}")

    print(f"Scheduler '{scheduler_type}' initialized successfully.")
    return config

In [5]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [6]:
# THE WHOLE TRAINING SETTINGS ARE HERE!

# GENERAL
BATCH_SIZE = 32
EPOCHS = 15
VAL_SPLIT = 0.1
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BACKBONE_FREEZING = False

# FL
NUM_CLIENTS = 100
NC = 50
LOCAL_EPOCHS = 4
NUM_ROUNDS = 8
FRACTION_FIT = 0.1
FRACTION_EVAL = 0.03
CLIENT_TYPE = clients.ClientType.TALOSPFEDEDIT
STRATEGY_TYPE = strategies.StrategiesType.YOGI
IID = False

# LOSS
SMOOTHING=0.1

# Optimizer Hyperparameters
LR = 0.00005
MOMENTUM = 0.9
WEIGHT_DECAY = 0.025
NESTEROV = False
BETAS = (0.9, 0.999)
EPSILON = 1e-8
ALPHA = 0.99
CENTERED = False
OPTIMIZER_TYPE  = clients.OptimizerType.SADAMW

# Scheduler Hyperparameters
T_MAX = LOCAL_EPOCHS
ETA_MIN = 0.0
STEP_SIZE = 30
GAMMA = 0.1
MILESTONES = [50, 75]
FACTOR = 0.1
PATIENCE = 5
THRESHOLD = 1e-4
TOTAL_ITERS = 10
LR_END = 0.0
SCHEDULER_TYPE  = "cosine"

TALOS_CONFIG = {
    "final_sparsity": 0.6,
    "num_batches": 3,
    "rounds": 4,
    "calibration_mode": "most_sensitive", # least_sensitive or most_sensitive
    "mode": "full", # full if want to include all layers, head for head only, pfededit for custom topk client selection
    "k": 3, #pfedit setting
}

# FedProx

MU = 0.1

# YOGI

ETA = 0.5         # Learning rate for Yogi updates
ETA_L = 0.25      # Learning rate for local updates
TAU = 0.1            # Adaptive learning parameter for Yogi
BETA_1 = 0.9         # Momentum term (default value from the original paper)
BETA_2 = 0.999       # Second momentum term (default value from the original paper)

# METAFEDAVG

INNER_LR = 0.01

#PFEDEDIT
LOCAL_EPOCHS_PFEDEDIT = 4
TOP_K_LAYERS = 3
#STOCHASTIC_FACTOR = 1
MAX_BATCHES = 4
PFEDEDIT_CONFIG = {
    "local epochs": LOCAL_EPOCHS_PFEDEDIT,
    "top_k_layers": TOP_K_LAYERS,
    "stochastic factor": "linear decrease",
    "max_batches": MAX_BATCHES
}

OPTIMIZER_CONFIG = build_optimizer_config(OPTIMIZER_TYPE)
SCHEDULER_CONFIG = build_scheduler_config(SCHEDULER_TYPE)

Optimizer 'OptimizerType.SADAMW' initialized successfully.
Scheduler 'cosine' initialized successfully.


In [7]:
pipeline = data_preprocessing.CIFAR100Pipeline(val_split=VAL_SPLIT, use_augment=True)
trainset, valset, testset = pipeline.run_pipeline()

trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
valloader = DataLoader(valset, batch_size=BATCH_SIZE)
testloader = DataLoader(testset, batch_size=BATCH_SIZE)

# Apply sharding
client_datasets_iid = data_utils.iid_split(trainset, NUM_CLIENTS)
client_datasets_noniid = data_utils.non_iid_split(trainset, NUM_CLIENTS, NC, 100)

print("Setup complete. IID and Non-IID splits created.")

100%|██████████| 169M/169M [00:18<00:00, 9.03MB/s]


Setup complete. IID and Non-IID splits created.


In [8]:
# Create model
def create_dino_vit_s16_for_cifar100(freezing=BACKBONE_FREEZING):
    model = timm.create_model("vit_small_patch16_224_dino", pretrained=True, num_classes=0)

    # Replace the head with CIFAR-100 classification head
    model.head = nn.Linear(model.num_features, 100)

    if freezing:
      # Freeze all parameters except head
      for param in model.parameters():
          param.requires_grad = False

      # Unfreeze only the head
      for param in model.head.parameters():
          param.requires_grad = True
    else:

      for param in model.parameters():
          param.requires_grad = True

      for param in model.head.parameters():
          param.requires_grad = True

    return model

In [9]:
model = create_dino_vit_s16_for_cifar100()

print(next(model.parameters()).device)
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable:,} / {total:,}")

initial_parameters = [val.cpu().numpy() for _, val in model.state_dict().items()]
flower_parameters = ndarrays_to_parameters(initial_parameters)

/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name vit_small_patch16_224_dino to current vit_small_patch16_224.dino.
  model = create_fn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/86.7M [00:00<?, ?B/s]

cpu
Trainable params: 21,704,164 / 21,704,164


In [10]:
!wandb login 89e5fee022a3a1cf86f958ee0b3dff6f2aa57aad

wandb: WARNING Using legacy-service, which is deprecated. If this is unintentional, you can fix it by ensuring you do not call `wandb.require('legacy-service')` and do not set the WANDB_X_REQUIRE_LEGACY_SERVICE environment variable.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [11]:
data_distribution = "IID_DATA" if IID else "NON_IID_DATA"
run_name = f"FEDERATED_{data_distribution}_TALOS_{datetime.now().strftime('%Y%m%d-%H%M%S')}"

if CLIENT_TYPE == clients.ClientType.TALOS:
    client_app = ClientApp(
        client_fn=clients.build_client_talos_fn(
            use_iid=IID,
            optimizer_type=OPTIMIZER_TYPE,
            optimizer_config=OPTIMIZER_CONFIG,
            scheduler_type=SCHEDULER_TYPE,
            scheduler_config=SCHEDULER_CONFIG,
            iid_partitions=client_datasets_iid,
            non_iid_partitions=client_datasets_noniid,
            model_fn=create_dino_vit_s16_for_cifar100,
            device=DEVICE,
            valset=valset,
            batch_size=BATCH_SIZE,
            local_epochs=LOCAL_EPOCHS,
            talos_config=TALOS_CONFIG
        )
    )
    client_config = None
elif CLIENT_TYPE == clients.ClientType.TALOSPROX:
    client_app = ClientApp(
        client_fn=clients.build_client_talos_prox_fn(
            use_iid=IID,
            optimizer_type=OPTIMIZER_TYPE,
            optimizer_config=OPTIMIZER_CONFIG,
            scheduler_type=SCHEDULER_TYPE,
            scheduler_config=SCHEDULER_CONFIG,
            iid_partitions=client_datasets_iid,
            non_iid_partitions=client_datasets_noniid,
            model_fn=create_dino_vit_s16_for_cifar100,
            device=DEVICE,
            valset=valset,
            batch_size=BATCH_SIZE,
            local_epochs=LOCAL_EPOCHS,
            talos_config=TALOS_CONFIG,
            mu=MU
        )
    )
    client_config = {"mu": MU}

elif CLIENT_TYPE == clients.ClientType.TALOSPFEDEDIT:
    client_app = ClientApp(
        client_fn=clients.build_client_fn_talos_pfededit(
            use_iid=IID,
            optimizer_type=OPTIMIZER_TYPE,
            optimizer_config=OPTIMIZER_CONFIG,
            scheduler_type=SCHEDULER_TYPE,
            scheduler_config=SCHEDULER_CONFIG,
            iid_partitions=client_datasets_iid,
            non_iid_partitions=client_datasets_noniid,
            model_fn=create_dino_vit_s16_for_cifar100,
            device=DEVICE,
            valset=valset,
            batch_size=BATCH_SIZE,
            local_epochs=LOCAL_EPOCHS,
            talos_config=TALOS_CONFIG,
            pfededit_config = PFEDEDIT_CONFIG
        )
    )
    client_config = {"mu": MU}

if STRATEGY_TYPE == strategies.StrategiesType.FEDAVG:
    strategy_config = None
elif STRATEGY_TYPE == strategies.StrategiesType.METAFEDAVG:
    strategy_config = {"inner_lr": INNER_LR}
else:
    strategy_config = {"eta": ETA, "eta_l": ETA_L, "tau": TAU, "beta_1": BETA_1, "beta_2": BETA_2,}


if not IID:
  DataConfig = {"use_iid": IID, "num_classes": NC}
else:
  DataConfig = {"use_iid": IID}


logger = FederatedWandBLogger(
    project_name="federated-learning-project",
    run_name=run_name,
    global_config={
        # Federated Learning Configuration
        "data_config": DataConfig,
        "local_epochs": LOCAL_EPOCHS,
        "batch_size": BATCH_SIZE,
        "num_clients": NUM_CLIENTS,
        "fraction_fit": FRACTION_FIT,
        "fraction_evaluate": FRACTION_EVAL,
        "num_rounds": NUM_ROUNDS,
        "backbone_freezing": BACKBONE_FREEZING,
        "client_type": CLIENT_TYPE.value,
        "client_config": client_config,
        "strategy_type": STRATEGY_TYPE.value,
        "strategy_config": strategy_config,
        "val_split": VAL_SPLIT,

        # Optimizer Configuration
        "optimizer": OPTIMIZER_TYPE.value,
        "optimizer_config": OPTIMIZER_CONFIG,

        # Scheduler Configuration
        "scheduler": SCHEDULER_TYPE,
        "scheduler_config": SCHEDULER_CONFIG,

        "talos_config": TALOS_CONFIG,

        "pfededit_config": PFEDEDIT_CONFIG


    }
)

if STRATEGY_TYPE == strategies.StrategiesType.FEDAVG:
    strategy = strategies.FedAvgStandard(
        logger=logger,
        initial_parameters=flower_parameters,
        fraction_fit=FRACTION_FIT,
        min_fit_clients=int(FRACTION_FIT*NUM_CLIENTS),
        min_evaluate_clients=int(FRACTION_EVAL*NUM_CLIENTS),
        fraction_evaluate=FRACTION_EVAL,
        min_available_clients=NUM_CLIENTS,
        evaluate_metrics_aggregation_fn = lambda metrics: {
            "accuracy": sum(num * m["val_accuracy"] for num, m in metrics) / sum(num for num, _ in metrics)
        }
    )
elif STRATEGY_TYPE == strategies.StrategiesType.METAFEDAVG:
    strategy = strategies.MetaFedAvg(
        logger=logger,
        inner_lr=INNER_LR,
        initial_parameters=flower_parameters,
        fraction_fit=FRACTION_FIT,
        min_fit_clients=int(FRACTION_FIT*NUM_CLIENTS),
        min_evaluate_clients=int(FRACTION_EVAL*NUM_CLIENTS),
        fraction_evaluate=FRACTION_EVAL,
        min_available_clients=NUM_CLIENTS,
        evaluate_metrics_aggregation_fn = lambda metrics: {
            "accuracy": sum(num * m["val_accuracy"] for num, m in metrics) / sum(num for num, _ in metrics)
        }
    )
else:
    strategy = strategies.FedYogiStandard(
        logger=logger,
        initial_parameters=flower_parameters,
        fraction_fit=FRACTION_FIT,
        min_fit_clients=int(FRACTION_FIT*NUM_CLIENTS),
        min_evaluate_clients=int(FRACTION_EVAL*NUM_CLIENTS),
        fraction_evaluate=FRACTION_EVAL,
        min_available_clients=NUM_CLIENTS,
        eta=ETA,
        eta_l=ETA_L,
        tau=TAU,
        beta_1=BETA_1,
        beta_2=BETA_2,
        evaluate_metrics_aggregation_fn = lambda metrics: {
            "accuracy": sum(num * m["val_accuracy"] for num, m in metrics) / sum(num for num, _ in metrics)
        }
    )

def server_fn(context: Context) -> ServerAppComponents:
    """Construct components that set the ServerApp behaviour.

    You can use the settings in `context.run_config` to parameterize the
    construction of all elements (e.g the strategy or the number of rounds)
    wrapped in the returned ServerAppComponents object.
    """
    config = ServerConfig(num_rounds=NUM_ROUNDS)
    return ServerAppComponents(strategy=strategy, config=config)


server_app = ServerApp(server_fn=server_fn)

/usr/local/lib/python3.11/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Currently logged in as: s348517giuseppe (s348517giuseppe-politecnico-di-torino) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/usr/local/lib/python3.11/dist-packages/wandb/analytics/sentry.py:259: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}  # noqa


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

if device.type == "cuda":
    backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 1.0}}

In [ ]:
run_simulation(
    server_app=server_app,
    client_app=client_app,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config
)

DEBUG:flwr:Asyncio event loop already running.
INFO :      Starting Flower ServerApp, config: num_rounds=8, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 10 clients (out of 100)
(pid=1815) 2025-05-23 18:42:26.212481: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=1815) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=1815) E0000 00:00:1748025746.317565    1815 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=1815) E0000 00:00:1748025746.341671    1815 cuda_bla

(ClientAppActor pid=1815) LOG: Initializing client with CID=4
(ClientAppActor pid=1815) 4-LOG: Data partition assigned to client 4 -> Non-IID


(ClientAppActor pid=1815) /usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name vit_small_patch16_224_dino to current vit_small_patch16_224.dino.
(ClientAppActor pid=1815)   model = create_fn(


(ClientAppActor pid=1815) 4-LOG: Model initialized for client 4
(ClientAppActor pid=1815) 4-LOG: Dataloaders initialized for client 4
(ClientAppActor pid=1815) 4-LOG: Initialized with FedProx μ = 0.1
(ClientAppActor pid=1815) Current Round: 1
(ClientAppActor pid=1815) Total Rounds: 8
(ClientAppActor pid=1815) 4-LOG: Round 1 | Dynamic Stochastic Factor: 1.0000
(ClientAppActor pid=1815) 4-LOG: Random Sample: 0.4437
(ClientAppActor pid=1815) 4-LOG: Stochastic Sampling Activated (stochastic_factor=1.0000), Full Model Training
(ClientAppActor pid=1815) 🟢 Pruning will be applied to the entire model.
(ClientAppActor pid=1815) 4-LOG: Starting TaLoS calibration in mode: full
(ClientAppActor pid=1815) 🔎 Starting multi-round calibration for mode 'full'.
(ClientAppActor pid=1815) 🌀 Calibration Round 1/4
(ClientAppActor pid=1815) 📝 Calculating Fisher Information on 3 batches...
(ClientAppActor pid=1815) ✅ Fisher Information Computation Completed.
(ClientAppActor pid=1815) 🌀 Calibration Round 2/4
(C

In [ ]:
final_parameters = strategy.latest_parameters

# Convert Flower parameters to list of numpy arrays
ndarrays = parameters_to_ndarrays(final_parameters)

# Load them into a PyTorch model
iid_model = create_dino_vit_s16_for_cifar100()
iid_model.to(device)
state_dict = OrderedDict(
    (key, torch.tensor(val)) for key, val in zip(iid_model.state_dict().keys(), ndarrays)
)
iid_model.load_state_dict(state_dict)

correct, total, loss_total = 0, 0, 0.0
criterion = torch.nn.CrossEntropyLoss()

with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = iid_model(images)
        loss = criterion(outputs, labels)

        loss_total += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss = loss_total / total
test_accuracy = correct / total

logger.log_global_metrics({
    "test_loss": test_loss,
    "test_accuracy": test_accuracy
}, round_number=NUM_ROUNDS)

model_save_path = "final_federated_model.pth"
torch.save(iid_model.state_dict(), model_save_path)

logger.log_model(iid_model, path=model_save_path)

logger.finish()

print(f"✅ Test Accuracy: {test_accuracy:.4f} | Test Loss: {test_loss:.4f}")